## Rozpoznawanie samochodów w czasie rzeczywistym z użyciem sieci neuronowych

---

W tym etapie użyje częściowo gotowego zbioru danych i użyje go do trenowania modelu.

Użyty zbiór danych: https://www.kaggle.com/datasets/sshikamaru/car-object-detection/data

Użyta architektura : https://github.com/ultralytics/yolov5

---

### Zacznę od wczytania potrzebnych bibliotek

In [ ]:
import torch
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os
!pip install ultralytics
from ultralytics import YOLO
import PIL
import shutil
from google.colab import userdata


Klonuje repozytorium yolov5

In [ ]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 17377, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 17377 (delta 43), reused 20 (delta 20), pack-reused 17313 (from 3)
Receiving objects: 100% (17377/17377), 16.25 MiB | 29.83 MiB/s, done.
Resolving deltas: 100% (11911/11911), done.


Pobieram z kaggla zbiór danych

In [ ]:
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')

In [ ]:
!kaggle datasets download -d sshikamaru/car-object-detection
!unzip car-object-detection

Dataset URL: https://www.kaggle.com/datasets/sshikamaru/car-object-detection
License(s): other
Archive:  car-object-detection.zip
  inflating: data/sample_submission.csv  
  inflating: data/testing_images/vid_5_25100.jpg  
  inflating: data/testing_images/vid_5_25120.jpg  
  inflating: data/testing_images/vid_5_25140.jpg  
  inflating: data/testing_images/vid_5_25160.jpg  
  inflating: data/testing_images/vid_5_25180.jpg  
  inflating: data/testing_images/vid_5_25200.jpg  
  inflating: data/testing_images/vid_5_25220.jpg  
  inflating: data/testing_images/vid_5_25240.jpg  
  inflating: data/testing_images/vid_5_25260.jpg  
  inflating: data/testing_images/vid_5_26320.jpg  
  inflating: data/testing_images/vid_5_26400.jpg  
  inflating: data/testing_images/vid_5_26420.jpg  
  inflating: data/testing_images/vid_5_26560.jpg  
  inflating: data/testing_images/vid_5_26580.jpg  
  inflating: data/testing_images/vid_5_26600.jpg  
  inflating: data/testing_images/vid_5_26620.jpg  
  inflating:

Chce pobrany zbiór danych doprowadzić do uporządowanej struktury

In [ ]:
try:
  os.rename('data', 'samochody')
except FileNotFoundError:
  print("nazwa katalogu juz zmieniona")

os.mkdir('samochody/images')


In [ ]:
try:
  os.rename('samochody/training_images', 'samochody/train')
  os.rename('samochody/testing_images', 'samochody/test')
except FileNotFoundError:
  print("nazwy katalogów juz zmienione")

try:
  shutil.move('samochody/train', 'samochody/images/')
  shutil.move('samochody/test', 'samochody/images/')
except FileNotFoundError:
  print("katalogi juz przeniesione")


Następnie musze przygotować zbiór danych w odpowiednim formacie do trenowania.

W tym przypadku bounding boxy dla danych uczących są gotowe, jedynie należy przeprowadzić transformację do odpowiedniego formatu.

In [ ]:
csv_path = "samochody/train_solution_bounding_boxes (1).csv"
df = pd.read_csv(csv_path)
df.head()

,image,xmin,ymin,xmax,ymax
0,vid_4_1000.jpg,281.259045,187.035071,327.727931,223.225547
1,vid_4_10000.jpg,15.163531,187.035071,120.329957,236.430180
2,vid_4_10040.jpg,239.192475,176.764801,361.968162,236.430180
3,vid_4_10020.jpg,496.483358,172.363256,630.020260,231.539575
4,vid_4_10060.jpg,16.630970,186.546010,132.558611,238.386422


Dla danych uczących:

In [ ]:
os.makedirs('samochody/labels/train')
for _, row in df.iterrows():
    it = "/images/train/"
    lt = "/labels/train/"
    image_path = "samochody" + it + row['image']
    xmin, ymin, xmax, ymax = row[['xmin', 'ymin', 'xmax', 'ymax']]

    img = PIL.Image.open(image_path)
    img_width, img_height = img.size

    x_center = ((xmin + xmax) / 2) / img_width
    y_center = ((ymin + ymax) / 2) / img_height
    width = (xmax - xmin) / img_width
    height = (ymax - ymin) / img_height

    txt_path = "samochody" + lt + row['image'].split('.')[0] + ".txt"
    with open(txt_path, 'a') as f:
        f.write(f"{0} {x_center} {y_center} {width} {height}\n")

Dla danych testowych bounding boxy stworzyłem używając label-studio, ponieważ nie ma ich w oryginalnym zbiorze. Etykiety umieściłem w moim repozytroium

In [ ]:
!git clone https://github.com/iochgsthsu/rozpoznawanie/

Cloning into 'rozpoznawanie'...
remote: Enumerating objects: 501, done.
remote: Counting objects: 100% (501/501), done.
remote: Compressing objects: 100% (487/487), done.
remote: Total 501 (delta 9), reused 497 (delta 8), pack-reused 0 (from 0)
Receiving objects: 100% (501/501), 3.08 MiB | 30.35 MiB/s, done.
Resolving deltas: 100% (9/9), done.


In [ ]:
try:
  shutil.move('rozpoznawanie/samochody/labels/test', 'samochody/labels/')
except FileNotFoundError:
  print("katalogi juz przeniesione")

In [ ]:
os.mkdir('datasets')
shutil.move('samochody', 'datasets/')
shutil.move('rozpoznawanie/dataset.yaml', 'datasets/')

'datasets/dataset.yaml'

Następnie tworze plik konfiguracyjny zbioru danych

In [ ]:
try:
  with open('datasets/dataset.yaml', "r", encoding="utf-8") as c:
    print(c.read())
except FileNotFoundError:
  print("Nie znaleziono pliku")

# moj_dataset.yaml
path: samochody/
train: images/train/
val: images/test/
nc: 1  
names: ['samochód']  # Nazwy klas


Wczytuje model ze wstępnie zainicjowanymi wagami.

In [ ]:
try:
  with open('yolov5/models/yolov5s.yaml', "r", encoding="utf-8") as y:
    print(y.read())
except FileNotFoundError:
  print("Plik nie został znaleziony")

model = YOLO("yolov5su.pt")


# Ultralytics 🚀 AGPL-3.0 License - https://ultralytics.com/license

# Parameters
nc: 80 # number of classes
depth_multiple: 0.33 # model depth multiple
width_multiple: 0.50 # layer channel multiple
anchors:
  - [10, 13, 16, 30, 33, 23] # P3/8
  - [30, 61, 62, 45, 59, 119] # P4/16
  - [116, 90, 156, 198, 373, 326] # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [
    [-1, 1, Conv, [64, 6, 2, 2]], # 0-P1/2
    [-1, 1, Conv, [128, 3, 2]], # 1-P2/4
    [-1, 3, C3, [128]],
    [-1, 1, Conv, [256, 3, 2]], # 3-P3/8
    [-1, 6, C3, [256]],
    [-1, 1, Conv, [512, 3, 2]], # 5-P4/16
    [-1, 9, C3, [512]],
    [-1, 1, Conv, [1024, 3, 2]], # 7-P5/32
    [-1, 3, C3, [1024]],
    [-1, 1, SPPF, [1024, 5]], # 9
  ]

# YOLOv5 v6.0 head
head: [
    [-1, 1, Conv, [512, 1, 1]],
    [-1, 1, nn.Upsample, [None, 2, "nearest"]],
    [[-1, 6], 1, Concat, [1]], # cat backbone P4
    [-1, 3, C3, [512, False]], # 13

    [-1, 1, Conv, [256, 1, 1]],
    [-1, 1, nn.Upsample, [None, 2, 

In [ ]:
model.info()

YOLOv5s summary: 153 layers, 9,153,152 parameters, 0 gradients, 24.2 GFLOPs


(153, 9153152, 0, 24.2129408)

Tworze plik konfiguracyjny trenowania

In [ ]:
config = {
    "data": "datasets/dataset.yaml",
    "mode": "train",
    "epochs": 100,
    "batch": 16,
#    "patience": 20,
    "imgsz": 640,
    # transfer learning
    "freeze": 10, # zamrazam wszystkie warstwy w backbone
 #   "lr0": 0.001,
    # augmentacja
    "hsv_h": 0.015,
    "hsv_s": 0.7,
    "hsv_v": 0.4,
    "degrees": 30,
    "translate": 0.1,
    "shear": 0.2,
    "flipud": 0.5,
    "fliplr": 0.5,
    "mosaic": 1.0,
    "mixup": 0.2,
    "scale": 0.5
}

In [ ]:
try:
    with open('datasets/cfg.yaml', 'w', encoding='utf-8') as f:
        yc = '\n'.join([f"{key}: {value}" for key, value in config.items()])
        f.write(yc)
except (FileNotFoundError, IsADirectoryError) as e:
    print(f"Błąd zapisu pliku: {e}")

In [ ]:
try:
  with open('datasets/cfg.yaml', "r", encoding="utf-8") as c:
    print(c.read())
except FileNotFoundError:
  print("Nie znaleziono pliku")

data: datasets/dataset.yaml
mode: train
epochs: 100
batch: 16
imgsz: 640
freeze: 10
hsv_h: 0.015
hsv_s: 0.7
hsv_v: 0.4
degrees: 30
translate: 0.1
shear: 0.2
flipud: 0.5
fliplr: 0.5
mosaic: 1.0
mixup: 0.2
scale: 0.5


Następnie przechodze do trenowania modelu

In [ ]:
results = model.train(cfg="datasets/cfg.yaml")

Ultralytics 8.3.108 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov5su.pt, data=datasets/dataset.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=10, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None

train: Scanning /content/datasets/samochody/labels/train.cache... 355 images, 646 backgrounds, 0 corrupt: 100%|██████████| 1001/1001 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/datasets/samochody/labels/test.cache... 175 images, 83 backgrounds, 0 corrupt: 100%|██████████| 175/175 [00:00<?, ?it/s]


Plotting labels to runs/detect/train3/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 69 weight(decay=0.0), 76 weight(decay=0.0005), 75 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train3
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

    100/100      2.47G     0.8829      0.423      1.022          9        640: 100%|██████████| 63/63 [00:16<00:00,  3.80it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:01<00:00,  4.48it/s]

                   all        175        156       0.94      0.788      0.872      0.325



100 epochs completed in 0.566 hours.
Optimizer stripped from runs/detect/train3/weights/last.pt, 18.5MB
Optimizer stripped from runs/detect/train3/weights/best.pt, 18.5MB

Validating runs/detect/train3/weights/best.pt...
Ultralytics 8.3.108 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv5s summary (fused): 84 layers, 9,111,923 parameters, 0 gradients, 23.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:01<00:00,  3.20it/s]


                   all        175        156      0.947      0.827      0.878      0.382
Speed: 0.2ms preprocess, 2.3ms inference, 0.0ms loss, 2.6ms postprocess per image
Results saved to runs/detect/train3


Zapisuje model

In [ ]:
model.save("moj_yolov5.pt")

Zipuje katalog runs

In [ ]:
folder_to_zip = '/content/runs'

output_zip_file = '/content/runs.zip'

try:
  shutil.make_archive(output_zip_file.replace('.zip', ''), 'zip', folder_to_zip)
except FileNotFoundError:
  print("Plik nie został znaleziony")


print(f"Folder {folder_to_zip} został spakowany do {output_zip_file}")

Folder /content/runs został spakowany do /content/runs.zip


---